Basically, segmentation is a process that partitions an image into regions. It is an image processing approach that allows us to separate objects and textures in images. Segmentation is especially preferred in applications such as remote sensing or tumor detection in biomedicine.

U-Net is more successful than conventional models, in terms of architecture and in terms pixel-based image segmentation formed from convolutional neural network layers. It’s even effective with limited dataset images. The presentation of this architecture was first realized through the analysis of biomedical images.

In [ ]:
import sys
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import os
from skimage.io import imread as imread
from PIL import Image
import imageio
import random

import tensorflow as tf
from pylab import rcParams

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn import metrics

from sklearn.model_selection import GridSearchCV

from keras.models import Model, load_model
from keras import optimizers, losses, activations, models

from keras.layers import Input, Dropout, concatenate, GlobalAveragePooling2D
from keras.layers import UpSampling2D
#from keras.layers.core import Lambda, RepeatVector, Reshape
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers import Convolution2D, ZeroPadding2D, MaxPooling2D, Cropping2D, Conv2D

from keras.layers.pooling import MaxPooling2D
#from keras.layers.merge import concatenate
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.callbacks import TensorBoard

from keras import backend as K
from keras.utils.np_utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator


from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, BatchNormalization
from keras import applications
from keras.applications import resnet50
from keras.optimizers import Adam

import warnings
warnings.filterwarnings("ignore")

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 
#Set numpy and Tensorflow random seed to mask sure experiment reproducible(only works in CPU mode).
from numpy.random import seed
seed(123)
from tensorflow import set_random_seed
set_random_seed(123)


import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import imageio

import matplotlib.pyplot as plt
%matplotlib inline

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
image_path = '../input/dataa/dataA/CameraRGB/'
mask_path = '../input/dataa/dataA/CameraSeg/'

image_list = os.listdir(image_path)
mask_list = os.listdir(mask_path)
image_list = [image_path+i for i in image_list]
mask_list = [mask_path+i for i in mask_list]

In [ ]:
N = 1
img = imageio.imread(image_list[N])
mask = imageio.imread(mask_list[N])
mask = np.array([max(mask[i, j]) for i in range(mask.shape[0]) for j in range(mask.shape[1])]).reshape(img.shape[0], img.shape[1])

fig, arr = plt.subplots(1, 2, figsize=(14, 10))
arr[0].imshow(img)
arr[0].set_title('Image')
arr[1].imshow(mask, cmap='Paired')
arr[1].set_title('Segmentation')

In [ ]:
img.shape

In [ ]:
mask.shape

In [ ]:
N = 121
img = imageio.imread(image_list[N])
mask = imageio.imread(mask_list[N])
print(mask.shape)
mask = np.array([max(mask[i, j]) for i in range(mask.shape[0]) for j in range(mask.shape[1])]).reshape(img.shape[0], img.shape[1])
print(mask.shape)

fig, arr = plt.subplots(1, 2, figsize=(14, 10))
arr[0].imshow(img)
arr[0].set_title('Image')
arr[1].imshow(mask, cmap='Paired')
arr[1].set_title('Segmentation')

In [ ]:
# 13 Classes from 0 to 12
set([z for i in mask for z in i])

In [ ]:
codes = ["Unlabeled",
          "Building",
          "Fence",
          "Other",
          "Pedestrian",
          "Pole",
          "Road line",
          "Road",
          "Sidewalk",
          "Vegetation",
          "Car",
          "Wall",
          "Traffic sign"]

In [ ]:
codes[6]

In [ ]:
len(image_list), len(mask_list)

## Road segmentation

In [ ]:
road = np.zeros((600, 800))
road[np.where(mask==7)[0], np.where(mask==7)[1]]=1
plt.imshow(road)

In [ ]:
from tqdm import tqdm
import sys
import dask
import dask.dataframe as dd

In [ ]:
len_images = len(image_list)
IMG_CHANNELS = 3
IMG_HEIGHT, IMG_WIDTH = 600, 800


number = 300

images = np.zeros((number, IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=np.uint8)
masks = np.zeros((number, IMG_HEIGHT, IMG_WIDTH, 1), dtype=np.uint8)


for n in range(number):

    if image_list[n].split('.')[-1] == 'png':

        #print(filename)
        img = imageio.imread(image_list[n])
        mask = imageio.imread(mask_list[n])
        
        mask_new = np.zeros((600, 800, 1), dtype=np.int8)
        for i in range(13):
            mask_new[np.where(mask==i)[0], np.where(mask==i)[1]]=i
       
        
        images[n] = img
        masks[n] = mask_new


In [ ]:
images.shape, masks.shape

In [ ]:
set(masks[2].flatten())

# Split Train / val / test

In [ ]:
np.random.seed(123)
shuffle_ids = np.array([i for i in range(len(masks))])
np.random.shuffle(shuffle_ids)

train_ids = shuffle_ids[:int(len(masks)*0.8)]
val_ids = shuffle_ids[int(len(masks)*0.8):int(len(masks)*0.8+30)]
test_ids = shuffle_ids[int(len(masks)*0.8+30):]

X_train, train_masks = images[train_ids], masks[train_ids]
X_val, val_masks = images[val_ids], masks[val_ids]
X_test, test_masks = images[test_ids], masks[test_ids]

In [ ]:
X_train.shape, X_val.shape, X_test.shape


# To categorical

In [ ]:
num_classes = 13

#y_train_categorize = to_categorical(train_masks, num_classes)
#y_val_categorize = to_categorical(val_masks, num_classes)
#y_test_categorize = to_categorical(test_masks, num_classes)

#train_masks.shape, y_train_categorize.shape


# Check if training data looks all right


In [ ]:
#ix = random.randint(0, number)
#plt.imshow(X_train[ix])
#plt.show()
plt.imshow(np.squeeze(train_masks[0]))
plt.show()

# Image generator

Un simple changement dans data augmentation change la val accuracy.
* Shear range = 0.5
* Zoom range = 0.3
* Horizontal flip = True
* Rescale = 1./255

In [ ]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.6,
        zoom_range=0.6,
        horizontal_flip=True)
        

In [ ]:
val_datagen = ImageDataGenerator(
        rescale=1./255)

In [ ]:
test_datagen = ImageDataGenerator(
        rescale=1./255)

In [ ]:

#train_generator = train_datagen.flow(X_train, y_train_categorize, batch_size=5, shuffle=True)
train_generator = train_datagen.flow(X_train, train_masks, batch_size=5, shuffle=True)

STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size


In [ ]:
#val_generator = val_datagen.flow(X_val, y_val_categorize, batch_size=5, shuffle=True)
val_generator = val_datagen.flow(X_val, val_masks, batch_size=5, shuffle=True)

STEP_SIZE_VAL=val_generator.n//val_generator.batch_size

In [ ]:
#test_generator = test_datagen.flow(X_test, y_test_categorize, batch_size=1, shuffle=True)
test_generator = test_datagen.flow(X_test, test_masks, batch_size=1, shuffle=False)

STEP_SIZE_TEST=test_generator.n//test_generator.batch_size

## Build U-Net with subtle changes

In [ ]:
from keras.models import Model, load_model
from keras.layers import Input
from keras.layers.core import Lambda, RepeatVector, Reshape
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras import backend as K

In [ ]:
# Build U-Net model
input_img = Input((IMG_HEIGHT, IMG_WIDTH, 3), name='img')


c1 = Conv2D(8, (3, 3), activation='relu', padding='same') (input_img)
c1 = Conv2D(8, (3, 3), activation='relu', padding='same') (c1)
p1 = MaxPooling2D((2, 2)) (c1)

c2 = Conv2D(16, (3, 3), activation='relu', padding='same') (p1)
c2 = Conv2D(16, (3, 3), activation='relu', padding='same') (c2)
p2 = MaxPooling2D((2, 2)) (c2)

c3 = Conv2D(32, (3, 3), activation='relu', padding='same') (p2)
c3 = Conv2D(32, (3, 3), activation='relu', padding='same') (c3)
p3 = MaxPooling2D((2, 2)) (c3)

c4 = Conv2D(64, (3, 3), activation='relu', padding='same') (p3)
c4 = Conv2D(64, (3, 3), activation='relu', padding='same') (c4)

u5 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same') (c4)
u5 = concatenate([u5, c3])
c6 = Conv2D(32, (3, 3), activation='relu', padding='same') (u5)
c6 = Conv2D(32, (3, 3), activation='relu', padding='same') (c6)

u7 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same') (c6)
u7 = concatenate([u7, c2])
c7 = Conv2D(16, (3, 3), activation='relu', padding='same') (u7)
c7 = Conv2D(16, (3, 3), activation='relu', padding='same') (c7)

u8 = Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same') (c7)
u8 = concatenate([u8, c1])
c8 = Conv2D(8, (3, 3), activation='relu', padding='same') (u8)
c8 = Conv2D(8, (3, 3), activation='relu', padding='same') (c8)

outputs = Conv2D(13, (1, 1), activation='sigmoid') (c8)

model = Model(inputs=[input_img], outputs=[outputs])


#model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy']) #, metrics=[mean_iou]) # The mean_iou metrics seens to leak train and test values...

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy']) #, metrics=[mean_iou]) # The mean_iou metrics seens to leak train and test values...
model.summary()

## Define callbacks settings

In [ ]:
tb = TensorBoard(log_dir='logs', write_graph=True)
mc = ModelCheckpoint(mode='max', filepath='camvid_model_150_epochs_checkpoint.h5', monitor='accuracy', save_best_only='True', save_weights_only='True', verbose=1)
es = EarlyStopping(mode='max', monitor='val_accuracy', patience=10, verbose=1)
callbacks = [tb, mc, es]


## Train and save the U-Net model

In [ ]:
batch_size = 5
steps_per_epoch = np.ceil(float(len(X_train) - round(0.1*len(X_train))) / float(batch_size))
steps_per_epoch

In [ ]:
validation_steps = (float((round(0.1*len(X_train)))) / float(batch_size))
validation_steps

In [ ]:
num_epochs = 200

result = model.fit_generator(train_generator, steps_per_epoch=18 ,
                validation_data = val_generator, 
                validation_steps = validation_steps, epochs=num_epochs, callbacks=callbacks)

model.save_weights("camvid_model_200_epochs.h5", overwrite=True)

#validation_steps=STEP_SIZE_VAL

# Learning curves

In [ ]:
plt.figure(1, figsize = (7,9)) 
    
plt.subplot(211)  
plt.plot(result.history['acc'])  
plt.plot(result.history['val_acc'])  
plt.title('model accuracy')  
plt.ylabel('accuracy')  
plt.xlabel('epoch')  
plt.ylim(0,1)
plt.legend(['train', 'valid']) 
#plt.grid(ls='--', c='C7')
   
plt.subplot(212)  
plt.plot(result.history['loss'])  
plt.plot(result.history['val_loss'])  
plt.title('model loss')  
plt.ylabel('loss')  
plt.xlabel('epoch')  
plt.legend(['train', 'valid']) 
#plt.grid(ls='--', c='C7')

plt.savefig('/kaggle/working/learning_curves_unet.png', bbox_inches='tight')

# Reloading best weights

In [ ]:
# load weights
model.load_weights("/kaggle/working/camvid_model_200_epochs.h5")
# Compile model (required to make predictions)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy']) #, metrics=[mean_iou]) # The mean_iou metrics seens to leak train and test values...
print("Created model and loaded weights from file")

# Prediction

 For each pixel the model predicts log probabilities for all classes.

In [ ]:
NUMBER = 152

test_path = '../input/datab/dataB/CameraRGB/'
test_list = os.listdir(test_path)
test_list = [test_path+i for i in test_list]
test = imageio.imread(test_list[NUMBER])

plt.imshow(test)
#my_preds = model.predict(np.expand_dims(test, 0))
#my_preds.shape

In [ ]:
predictions = model.predict_generator(test_generator,steps = STEP_SIZE_TEST)
#predictions = model_new.predict_generator(test_generator,steps = STEP_SIZE_TEST)

np.shape(predictions)

# Vizualizing predictions

In [ ]:
#single_layer = np.argmax(predictions[0], axis=-1)
#print(np.shape(single_layer))
#output = np.zeros(predictions.shape[1:3]+(1,) )
#for k in range(13):
#    output[single_layer==k] = k
#np.shape(output)

In [ ]:
def onehot_to_rgb(onehot):
    '''Function to decode encoded mask labels
        Inputs: 
            onehot - one hot encoded image matrix (height x width x num_classes)
            colormap - dictionary of color to label id
        Output: Decoded RGB image (height x width x 3) 
    '''
    single_layer = np.argmax(onehot, axis=-1)
    #print(np.shape(single_layer))
    output = np.zeros((600, 800, 1))
    for k in range(13):
        output[single_layer==k] = k
    #np.shape(output)
    return np.uint8(output)

In [ ]:
np.shape(X_test[0]), np.shape(test_masks[0]), np.shape(onehot_to_rgb(predictions[0])), np.shape(output)


In [ ]:
for i in range(0,np.shape(predictions)[0]):
#for i in range(5):
    
    fig = plt.figure(figsize=(20,8))
    
    ax1 = fig.add_subplot(1,3,1)
    ax1.imshow(X_test[i])
    ax1.title.set_text('Actual frame')
    ax1.grid(b=None)
    
    
    ax2 = fig.add_subplot(1,3,2)
    ax2.set_title('Ground truth labels')
    ax2.imshow(np.squeeze(test_masks[i]))
    ax2.grid(b=None)
    
    ax3 = fig.add_subplot(1,3,3)
    ax3.set_title('Predicted labels')
    ax3.imshow(np.squeeze(onehot_to_rgb(predictions[i])))
    ax3.grid(b=None)
    
    plt.show()

Applitcation aux images sat:

1. Comment conserver les positions géographiques des pixels
2. Comment gérer les GeoTiff

# Infering

In [ ]:
def infering(array):
    new = np.zeros((len(array),), dtype=np.uint8)
    index = 0
    
    for i in array:
        
        if i < 0.5:
            new[index] = 0
        elif i >= 0.5 and i < 1.5:
            new[index] = 1
        elif i >= 1.5 and i < 2.5:
            new[index] = 2
        elif i < 3.5 and i >= 2.5:
            new[index] = 3
        elif i < 4.5 and i >= 3.5:
            new[index] = 4
        elif i < 5.5 and i >= 4.5:
            new[index] = 5
        elif i < 6.5 and i >= 5.5:
            new[index] = 6
        elif i < 7.5 and i >= 6.5:
            new[index] = 7
        elif i < 8.5 and i >= 7.5:
            new[index] = 8
        elif i < 9.5 and i >= 8.5:
            new[index] = 9
        elif i < 10.5 and i >= 9.5:
            new[index] = 10
        elif i < 11.5 and i >= 10.5:
            new[index] = 11
        elif i < 12.5 and i >= 11.5:
            new[index] = 12
        index += 1
    return new